In [4]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import sys


print("Python version:", sys.version)
print("cv2 version:", cv2.__version__)
print("mediapipe version", mp.__version__)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)


Python version: 3.8.18 | packaged by conda-forge | (default, Oct 10 2023, 15:46:56) 
[Clang 16.0.6 ]
cv2 version: 4.8.1
mediapipe version 0.10.7
pandas version: 2.0.3
numpy version: 1.24.4


In [5]:
## Setting Class number and Hyperparameters
class_num=0     # you must change class_num when gathering geture.
max_num_hands=1  # Number of hands recognized



gesture={
    0 : 'Right',
    1 : 'Left',
    2 : 'Turn Clockwise',
    3 : 'Turn Anticlockwise'

}


In [6]:
# Mediapipe hands model

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=max_num_hands,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
# if click left mouse button gathering data is start and end.


# def click(event,x,y,flags,param):
#     global data, file
#     if event ==cv2.EVENT_LBUTTONDOWN:
#         file = np.vstack((file, data))
#         print(file.shape)


# cv2.namedWindow('Dataset')  # make window
# cv2.setMouseCallback('Dataset',click) # trigger mouse event 

In [4]:
# make Data set code

# 만약 기존의 데이터 파일에서 붙여서 계속 데이터를 모으고 싶다면, 아래 파일명만 바꾸면 됨.
# default_array = np.genfromtxt('./data/start_file.csv', delimiter=',')

# 기존의 데이터 없이 데이터 수집하는 경우.
default_array=np.array(range(100),dtype='float64')

webcam = cv2.VideoCapture(0)  # 몇번째 카메라인지 설정.

while webcam.isOpened():  # webcap 열려있는지 확인.
    seq=[]
    status, frame =webcam.read()

    if not status:
        continue

    frame=cv2.flip(frame,1)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:

        # 손에 대한 정보 실시간(프레임 느낌)으로 계속 가져옴.
        # res 에 21개의 (x,y,z) 정보가 들어있음( 각각 관절의 위치 값들)
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))

            
            # 21개의 관절 포인트에 대해 joint 할당함.
            for j, lm in enumerate(res.landmark):
                
                # x ,y, z(distance between camera and hand)
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility] # [21,4]
            

            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,4]
            
            
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]   #[20,4]
             

            # Get angle using arcos of dot product
            # angle [15,1]
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) 
            


            angle = np.degrees(angle) # Convert radian to degree
            angle = np.array([angle], dtype=np.float32)
            
            #  'angle + class_num'
            #   angle_label [16,1]
            angle_label = np.append(angle, class_num) 
           
            # joint[21,4] + angle_label[16,1] 
            # joint_angle_label [100,]
            joint_angle_label = np.concatenate([joint.flatten(), angle_label])



            mp_drawing.draw_landmarks(frame, res, mp_hands.HAND_CONNECTIONS)

            

            # np.array() 형태로 계속 붙음.(넘파이 형태임.)
            seq.append(joint_angle_label)  
            

        data=np.array(seq)
        default_array = np.vstack((default_array, data))
        # print('file shape : ',default_array.shape)

            
    
    cv2.imshow('Dataset', frame)
    if cv2.waitKey(1) == ord('q'):
        break


# save file
np.savetxt('./main_data/data_gesture_'+gesture[class_num]+'.csv', default_array[1:,:], delimiter=',')